In [1]:
import my_toolbox as tb
import numpy as np
import calibration as calib
import main
import importlib

importlib.reload(main)

main_path = "C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/"
my_lab_fe_grid = np.log(np.array([5.0, 10.0, 15.0, 20.0]))
myPars = main.pars_factory(main_path, my_lab_fe_grid = my_lab_fe_grid) 




Using default health transition matrix


In [14]:
from scipy.stats import norm
from typing import Tuple
from math import erf, sqrt, exp, pi
from numba import njit

def Taucheniid(sigma:float, num_grid_points: int, Nsd: int=3, mean:float=0.0, state_grid: np.ndarray=np.zeros(1))->Tuple[np.ndarray, np.ndarray]:
    """
    This function uses the method of Tauchen (1986) to approximate a continuous iid Normal process.

    Normal process: ε~N(0, σ**2).

    INPUTS:  -σ: SD of innovation in AR(1) process
             -S: number of gridpoints
             -Nsd: number of SD from mean for grid to span

    OUTPUTS: -state_grid, grid of state variable s
             -probs, grid of probabilities for each state
    """
    # compute grid over state s and the half-distance between gridpoints, δ
    if len(state_grid) == 1:
        state_grid = np.linspace(mean - Nsd * sigma, mean + Nsd * sigma, num_grid_points)
    δ = (state_grid[-1] - state_grid[0]) / (num_grid_points - 1) / 2

    # compute cumulative probabilities of state_grid
    probscum = np.ones(num_grid_points)
    for s in range(num_grid_points - 1):
        probscum[s] = norm.cdf(state_grid[s] + δ, loc=mean, scale=sigma)

    # compute probabilities of state_grid
    probs = probscum
    probs[1:] = probscum[1:] - probscum[:-1]

    return probs, state_grid


@njit
def normal_cdf_numba(x, mean=0.0, sigma=1.0):
    """
    Numba-compatible approximation for the CDF of the normal distribution.
    Uses the error function (erf) to compute the CDF.
    """
    return 0.5 * (1 + erf((x - mean) / (sigma * sqrt(2))))

@njit
def Taucheniid_numba(sigma: float, num_grid_points: int, Nsd: int = 3, mean: float = 0.0, state_grid: np.ndarray = np.zeros(1)) -> Tuple[np.ndarray, np.ndarray]:
    """
    Numba-compatible version of the Taucheniid function to approximate a continuous iid Normal process.

    Normal process: ε ~ N(0, σ**2).

    INPUTS:  - σ: SD of innovation in AR(1) process
             - num_grid_points: number of grid points
             - Nsd: number of standard deviations from the mean for grid to span

    OUTPUTS: - state_grid: grid of state variable s
             - probs: grid of probabilities for each state
    """
    # Compute grid over state s and the half-distance between grid points (δ)
    if len(state_grid) == 1:
        state_grid = np.linspace(mean - Nsd * sigma, mean + Nsd * sigma, num_grid_points)
    δ = (state_grid[-1] - state_grid[0]) / (num_grid_points - 1) / 2

    # Compute cumulative probabilities of state_grid using normal_cdf
    probscum = np.ones(num_grid_points)
    for s in range(num_grid_points - 1):
        probscum[s] = normal_cdf_numba(state_grid[s] + δ, mean=mean, sigma=sigma)

    # Compute probabilities of state_grid
    probs = probscum.copy()  # Copy to avoid modifying in place
    probs[1:] = probscum[1:] - probscum[:-1]

    return probs, state_grid


In [15]:
importlib.reload(calib)
mu = 12.0
sigma = 2.035

mu_mom_targ = 12.5
sigma_mom_targ = 3.5
# my_grid = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# my_grid = np.log(np.array([5.0, 10.0, 15.0, 20.0]))
my_grid = np.array([5.0, 10.0, 15.0, 20.0])
# my_grid = np.log(my_grid)

tauch_weights, tauch_grid = Taucheniid(sigma, len(my_grid), Nsd = 3, mean = mu, state_grid = my_grid)
tauch_weights_numba, tauch_grid_numba = Taucheniid_numba(sigma, len(my_grid), Nsd = 3, mean = mu, state_grid = my_grid)

# print tauch results
myPars.lab_fe_weights = tauch_weights 
mean = np.sum(tauch_weights*tauch_grid)
variance = np.sum(tauch_weights*(tauch_grid - mean)**2)
print(f"Tauch weights: {tauch_weights}")
print(f"Tauchen grid: {tauch_grid}")
print(f"Tauch mean: {mean}")
print(f"Tauch variance: {variance}")


# print tauch numba results
myPars.lab_fe_weights = tauch_weights_numba
mean = np.sum(tauch_weights_numba*tauch_grid_numba)
variance = np.sum(tauch_weights_numba*(tauch_grid_numba - mean)**2)
print(f"Tauch weights numba: {tauch_weights_numba}")
print(f"Tauchen grid numba: {tauch_grid_numba}")
print(f"Tauch mean numba: {mean}")
print(f"Tauch variance numba: {variance}")

# compute model moments to match
my_w0_mu_mom = calib.w0_mu_moment(myPars)
print(f"Given mu: {mu}, model mu moment: {my_w0_mu_mom}, target mu moment: {mu_mom_targ}")
my_w0_sigma_mom = calib.w0_sigma_moment(myPars)
print(f"Given sigma: {sigma}, model sigma moment: {my_w0_sigma_mom}, target sigma moment: {sigma_mom_targ}")

# compute model moments to match
myPars.lab_fe_weights = tauch_weights_numba
my_w0_mu_mom = calib.w0_mu_moment(myPars)
print(f"Given mu: {mu}, model mu moment: {my_w0_mu_mom}, target mu moment: {mu_mom_targ}")
my_w0_sigma_mom = calib.w0_sigma_moment(myPars)
print(f"Given sigma: {sigma}, model sigma moment: {my_w0_sigma_mom}, target sigma moment: {sigma_mom_targ}")


Tauch weights: [0.01350746 0.5835354  0.39951823 0.00343891]
Tauchen grid: [ 5. 10. 15. 20.]
Tauch mean: 11.964442972449088
Tauch variance: 6.81049716158292
Tauch weights numba: [0.01350746 0.5835354  0.39951823 0.00343891]
Tauchen grid numba: [ 5. 10. 15. 20.]
Tauch mean numba: 11.964442972449088
Tauch variance numba: 6.81049716158292
Given mu: 12.0, model mu moment: 13.730126231302608, target mu moment: 12.5
Given sigma: 2.035, model sigma moment: 3.462460905275781, target sigma moment: 3.5
Given mu: 12.0, model mu moment: 13.730126231302608, target mu moment: 12.5
Given sigma: 2.035, model sigma moment: 3.462460905275781, target sigma moment: 3.5
